In [2]:
import os
import pandas as pd
import timeit
import numpy as np
import glob, pickle, scipy.optimize, scipy.special
from pathlib import Path

from sklearn.metrics import log_loss

#path = Path('/kaggle/input/tabular-playground-series-nov-2022/')
path = Path('C:/Users/incar/Downloads/tabular-playground-series-nov-2022/')


### Read files and info

In [6]:
submission = pd.read_csv(path / 'sample_submission.csv', index_col='id')
labels = pd.read_csv(path / 'train_labels.csv', index_col='id')

# the ids of the submission rows (useful later)
sub_ids = submission.index

# the ids of the labeled rows (useful later)
gt_ids = labels.index 

# list of files in the submission folder
subs = sorted(os.listdir(path / 'submission_files'))

In [5]:
REBUILD_DATA = False
path = Path(path)
submission = pd.read_csv(path / 'sample_submission.csv', index_col='id')
    
if REBUILD_DATA:
    labels = pd.read_csv(path / 'train_labels.csv', index_col='id')

    # the ids of the submission rows (useful later)
    sub_ids = submission.index

    # the ids of the labeled rows (useful later)
    gt_ids = labels.index 

    # list of files in the submission folder
    subs = sorted(os.listdir(path / 'submission_files')) #sorted in order of performance
    
    dfs = [pd.read_csv(path / 'submission_files' / sub)['pred'] for sub in subs]
    df = pd.concat(dfs, axis=1)
    df.columns = [f'sub_{i+1:04}' for i in range(len(df.columns))]
    df['label'] = labels
    df.to_feather('/tps.ftr', index=None)
else:
    df = pd.read_feather('tps.ftr')
    
df.head()

,sub_0001,sub_0002,sub_0003,sub_0004,sub_0005,sub_0006,sub_0007,sub_0008,sub_0009,sub_0010,...,sub_4992,sub_4993,sub_4994,sub_4995,sub_4996,sub_4997,sub_4998,sub_4999,sub_5000,label
0,0.709336,0.799007,0.851891,0.537158,0.623930,0.705970,0.503437,0.633185,0.641550,0.666604,...,0.750250,0.663370,0.739333,0.822384,0.749498,0.729800,0.867847,0.745888,0.787,0.0
1,0.452988,0.364453,0.567582,0.354468,0.513818,0.584119,0.454809,0.238501,0.472171,0.522314,...,0.794052,0.721298,0.804369,0.620626,0.733606,0.816942,0.814229,0.598331,0.547,1.0
2,0.675462,0.842260,0.800013,0.525229,0.692071,0.715418,0.651008,0.609124,0.691198,0.609994,...,0.779859,0.865657,0.828493,0.763010,0.802883,0.806891,0.896058,0.855776,0.667,1.0
3,0.481046,0.577118,0.683032,0.541356,0.630088,0.664514,0.413373,0.508210,0.526140,0.584565,...,0.799698,0.800130,0.716604,0.603779,0.708499,0.844837,0.853057,0.850657,0.622,1.0
4,0.957339,0.910337,0.917322,0.874487,0.787595,0.854273,0.843846,0.876749,0.821128,0.913054,...,0.900150,0.960911,0.906037,0.961240,0.935608,0.889757,0.978505,0.953681,0.934,0.0


### Read in the first submission file (best scoring on labeled rows)

In [4]:
s0 = pd.read_csv(path / 'submission_files' / subs[0], index_col='id')

score = log_loss(labels, s0.loc[gt_ids])

# Notice the score of the labeled rows matches the file name
print(subs[0], f'{score:.10f}')

0.6222863195.csv 0.6222863195


### Same for second submission file

In [16]:
s1 = pd.read_csv(path / 'submission_files' / subs[1], index_col='id')

score = log_loss(labels, s1.loc[gt_ids])

print(subs[1], f'{score:.10f}')

0.6223807245.csv 0.6223807245


### Blending the two files

Blending `s0` and `s1` gives a local score of 0.60497, which is an improvement from the input files (0.622...)


In [6]:
blend = (s0 + s1) / 2

score = log_loss(labels, blend.loc[gt_ids])

print(f'blend score: {score:.10f}')

blend score: 0.6049715910


### How does the blend do on the Leaderboard?

If you submit the unlabeled rows of `s0` (e.g., 20,000 - 39,000) to the leaderboard, it scores 0.61863. Likewise, `s1` scores 0.62335. How does the blend do?

#### It scores 0.60454, which is an improvement!

In [8]:
blend.loc[sub_ids].to_csv('blend.csv')

# Data check

In [8]:
df['label'].value_counts()

0.0    10000
1.0    10000
Name: label, dtype: int64

In [9]:
df_train = df[df['label'].notna()]
df_test = df[df['label'].isna()]
y_train = df_train['label']
y_test = df_test['label']
df_train = df_train.drop(columns=['label'])
df_test = df_test.drop(columns=['label'])
X_train = df_train.values
X_test = df_test.values
loglosses = np.array([log_loss(y_train, df_train[c]) for c in df_train.columns])

## Top 100 average

In [10]:
topn = 100
df_pred_avg = df_train[df_train.columns[:topn]].mean(axis=1)
log_loss(y_train, df_pred_avg)

0.6023252766809227

In [25]:
df_100_avg = pd.DataFrame(df_pred_avg, columns = ["pred"]).set_index(sub_ids)